In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets.mnist import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from collections import OrderedDict

In [2]:
data_train=MNIST('../data/pytorch_data/mnist',download=True,transform=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
]))
data_test=MNIST('../data/pytorch_data/mnist',train=False,download=True,transform=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
]))

In [3]:
train_iter=DataLoader(data_train,batch_size=256,shuffle=True,num_workers=4)
test_iter=DataLoader(data_test,batch_size=256,num_workers=4)

In [ ]:
def rotate_(x,weight):
    sort=x.reshape(-1).argsort()
    max=sort[-1]
    if max==4:
        max=sort[-2]
    r=max//3
    l=max%3
    if r>l:
        degree=-(r*45+l*45)
    else:
        degree=r*45+l*45
    weight=Image.fromarray(weight)
    weight_rotate=weight.rotate(degree)
    return np.array(weight_rotate)

In [15]:
import torch.nn.functional as F
class Conv(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(Conv,self).__init__()
        kernel=torch.randn(out_channels,in_channels,3,3)
        kernel=torch.FloatTensor(kernel)
        self.weight=nn.Parameter(data=kernel,requires_grad=True)
    def forward(self,x):
        x=F.conv2d(x,self.weight)
        return x ,self.weight

In [26]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_iters=iter(train_iter)
train_data=next(train_iters)
images,labels=train_data[0].to(device),train_data[1].to(device)
net=Conv(1,6).cuda()
out,weight=net(images)

torch.Size([6, 1, 3, 3])

In [6]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5,self).__init__()
        self.fc=nn.Sequential(OrderedDict([
            ('fc6',nn.Linear(120*16,84)),
            ('relu6',nn.ReLU()),
            ('fc7',nn.Linear(84,10))
        ]))
        self.convnet=nn.Sequential(OrderedDict([
            ('c1',Conv(1,6)),
            ('relu1',nn.ReLU()),
            ('s2',nn.MaxPool2d(kernel_size=(2,2),stride=2)),
            ('c3',Conv(6,16)),
            ('relu3',nn.ReLU()),
            ('s4',nn.MaxPool2d(kernel_size=(2,2),stride=2)),
            ('c5',Conv(16,120)),
            ('relu5',nn.ReLU())
        ]))
    def  forward(self,img):
        output=self.convnet(img)
        output=output.view(-1,120*16)
        output=self.fc(output)
        return output

In [7]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net=LeNet5()
net=net.to(device)
loss=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [8]:
def accuracy(data_iter,net):
    acc=0.
    for i ,(images,labels) in enumerate(data_iter):
        images,labels=images.to(device),labels.to(device)
        output=net(images)
        acc+=(output.argmax(1)==labels).cpu().numpy().mean()
    acc/=len(data_iter)
    return acc

In [9]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
def train(num_epoch):
    for epoch in range(1,num_epoch):
        for i ,(images,labels) in enumerate(train_iter):
            images,labels=images.to(device),labels.to(device)
            optimizer.zero_grad()
            l=loss(net(images),labels)
            l.backward()
            optimizer.step()
        if epoch %5==0:
            train_acc,test_acc=accuracy(train_iter,net),accuracy(test_iter,net)
            print('train_acc:%f,test_acc:%f'%(train_acc,test_acc))

In [10]:
def test():
    imgs,labels=next(iter(test_iter))
    imgs=imgs.to(device)
    pred=net(imgs).argmax(1) 
    print(pred[:10].cpu(),labels[:10])

In [11]:
if __name__=='__main__':
    num_epoch=11
    train(num_epoch)
    test()

train_acc:0.947767,test_acc:0.949414
train_acc:0.960262,test_acc:0.958496
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9]) tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])


In [1]:
import numpy as np
from PIL import Image
from math import sqrt

In [2]:
def rotate_(x,weight):
    sort=x.reshape(-1).argsort()
    max=sort[-1]
    if max==4:
        max=sort[-2]
    r=max//3
    l=max%3
    if r>l:
        degree=-(r*45+l*45)
    else:
        degree=r*45+l*45
    x=Image.fromarray(x)
    x_rotate=x.rotate(degree)
    return np.array(x_rotate)

In [3]:
def rotate(features,width):
    temp=[]
    for i in range(width-2):
        for j in range(width-2):
            x=features[i:i+3,j:j+3]
            temp.append(rotate_(x))
    rows=int(sqrt(len(temp)))
    temp1=[]
    for i in range(rows):
        temp1.append(np.concatenate(temp[i*rows:i*rows+rows],1))
    features_rotate=np.concatenate(temp1,0)
    return features_rotate

In [4]:
width=4
features=np.uint8(np.random.rand(width**2).reshape(width,width)*100)
print(features)
features_rotate=rotate(features,width) 
print(features_rotate)

[[24 33 82 41]
 [15  5 33 45]
 [88 85 74 16]
 [13 79  1 79]]
[[88 15 24 85  5 33]
 [85  5 33 74 33 82]
 [74 33 82 16 45 41]
 [88 15  5 85  5 33]
 [13 85 33 79 74 45]
 [79  1 74  1 79 16]]
